In [1]:
# Styling notebook
from IPython.core.display import display, HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

C:\Users\Renata\AppData\Local\Temp\ipykernel_11708\1617930524.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from IPython.display import display, HTML
display(HTML('<style>.container {width: 100% !important}</style>'))

# Inicio

In [3]:
# <span style="border:1px solid blue;background-color:yellow;color:blue;font-family:Courier;font-size:1.0em"> Inicio </span>

## Links uteis 

- Cores em graficos plotly: https://plotly.com/python/discrete-color/
- https://www.selecolor.com/en/recommended-color-palette/

## Importação de libs

In [4]:
# para ler arquivo excel (xls)
# conda install -c anaconda xlrd

import datetime
from datetime import datetime
import os

# Analise
import pandas as pd 
import numpy as np 

#Visualization
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
import plotly
import plotly.express as px

## Configurações

In [5]:
# para habilitar rolagem horizontal do dataframe 
# caso não funcione no Firefox
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [6]:
# formatação de mensagens de print
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
#print(color.BOLD + 'Hello World !' + color.END)

## Parâmetros

In [7]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
pd.set_option('max_colwidth', 1000) # tamanho das colunas para exibição do conteúdo de textos longos

np.set_printoptions(threshold=1000)

# Formatar numeros float (2 casas decimais)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# Formatações para gráficos
sns.set(style="darkgrid") #cor da area dos plots
figsize = (10,10)

# Definição de funções

## checar_df

Para um determinado dataframe, são verificados:
- total de registros;
- quantidade e tipos de colunas;
- quantidade de valores únicos;
- quantidade de valores nulos (para identificar a necessidade de pre processamento dos dados).

In [8]:
def checar_df(df):
    print(color.BOLD  + color.RED + 'Informações do dataframe: ' + color.END)
    df.info()
    
    print(color.BOLD + color.RED + "\nNúmero de observacoes na base de dados: " + color.END, df.shape[0])
    print('Total de ', df.shape[1], 'Colunas: ', df.columns.values.tolist())
    
    print(color.BOLD + color.RED + "\nNúmero de valores unicos (Não inclui NaN): " + color.END)
    display(df.nunique(dropna=True).to_frame(name='Contagem Unicos').sort_index())
    
    print(color.BOLD + color.RED + "Número de valores nulos:" + color.END)
    display(df.isnull().sum().sort_values(ascending=False).to_frame(name='Contagem Nulos').sort_index())

## define_tp_cat_adm

In [9]:
def define_tp_cat_adm(x):
    if x == 1:   return 'Pública Federal'
    elif x == 2: return 'Pública Estadual'
    elif x == 3: return 'Pública Municipal'
    elif x == 4: return 'Privada com fins lucrativos'
    elif x == 5: return 'Privada sem fins lucrativos'
    elif x == 6: return 'Privada - Particular em sentido estrito'
    elif x == 7: return 'Especial'        
    elif x == 8: return 'Privada comunitária'        
    elif x == 9: return 'Privada confessional'        
    else: return 'não informado'

## define_tp_org_acad

In [10]:
def define_tp_org_acad(x):
    if x == 1: return 'Universidade'    
    elif x == 2: return 'Centro Universitário'    
    elif x == 3: return 'Faculdade'
    elif x == 4: return 'Instituto Federal de Educação, Ciência e Tecnologia'
    elif x == 5: return 'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

## define_tp_org_principal

In [11]:
def define_tp_org_principal(x):
    if x == 1: return 'Faculdade'               #'Universidade'    
    elif x == 2: return 'Centro Universitário'  #'Centro Universitário'    
    elif x == 3: return 'Faculdade'             #'Faculdade'
    elif x == 4: return 'Outros'  # 'Instituto Federal de Educação, Ciência e Tecnologia' 
    elif x == 5: return  'Outros' #'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

# Carga dos Dados 

## Carga dados IBGE

Dados do IBGE se encontram em 3 csvs - portanto, ambos serão carregados em diferentes dataframes para depois terem algumas colunas de interesse serem adicionadas ao dataframe principal (df_ies).

In [12]:
# definir os tipos de algumas colunas
# para facilitar operações de merge
list_col_string = ['COD_IBGE','COD_SIAFI','COD_UE_TSE_2016','CAPITAL'] 
dict_dtype = {column : 'str'  for column in list_col_string}

ibge1 = pd.read_csv('./dados/ibge.csv', sep=';', dtype = dict_dtype, low_memory=False)

# renomear colunas 
ibge1 = ibge1.rename(columns = {'POP2013':'POP_IBGE_2013'})

ibge1.info()
ibge1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COD_IBGE           5570 non-null   object 
 1   COD_SIAFI          5569 non-null   object 
 2   NOME_MUNICIPIO     5570 non-null   object 
 3   Nome_Simplificado  5570 non-null   object 
 4   SIGLA_UF           5570 non-null   object 
 5   NOME_UF            5570 non-null   object 
 6   LAT                5570 non-null   float64
 7   LNG                5570 non-null   float64
 8   POP_IBGE_2013      5570 non-null   int64  
 9   CAPITAL            5570 non-null   object 
 10  ALT                5570 non-null   float64
 11  REGIAO             5570 non-null   object 
 12  COD_UE_TSE_2016    5568 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 565.8+ KB


,COD_IBGE,COD_SIAFI,NOME_MUNICIPIO,Nome_Simplificado,SIGLA_UF,NOME_UF,LAT,LNG,POP_IBGE_2013,CAPITAL,ALT,REGIAO,COD_UE_TSE_2016
0,1100015,0033,Alta Floresta D'Oeste,ALTA FLORESTA DOESTE,RO,Rondônia,-11.94,-62.00,25728,0,337.74,N,00310
1,1100023,0007,Ariquemes,ARIQUEMES,RO,Rondônia,-9.91,-63.03,101269,0,138.69,N,00078
2,1100031,0037,Cabixi,CABIXI,RO,Rondônia,-13.50,-60.54,6495,0,236.06,N,00450
3,1100049,0009,Cacoal,CACOAL,RO,Rondônia,-11.43,-61.44,85863,0,177.45,N,00094
4,1100056,0027,Cerejeiras,CEREJEIRAS,RO,Rondônia,-13.20,-60.82,18041,0,262.81,N,00272


In [13]:
# definir os tipos de algumas colunas
list_col_string = ['COD_IBGE','COD. UF','COD. MUNIC'] 
dict_dtype = {column : 'str'  for column in list_col_string}

# dados atualizados de População (Censo 2022)
ibge2 = pd.read_csv('./dados/POP2022_Municipios_20230622.csv', 
                    sep='|', 
                    encoding='iso-8859-1',
                    dtype = dict_dtype, 
                    low_memory=False)

# renomear colunas 
ibge2 = ibge2.rename(columns = {'POPULAÇÃO':'POP_IBGE_2022'})

# converter para int
ibge2['POP_IBGE_2022'] = ibge2['POP_IBGE_2022'].astype(int)

ibge2.info()
ibge2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   UF                 5570 non-null   object
 1   COD. UF            5570 non-null   object
 2   COD. MUNIC         5570 non-null   object
 3   COD_IBGE           5570 non-null   object
 4   NOME DO MUNICÍPIO  5570 non-null   object
 5   POP_IBGE_2022      5570 non-null   int32 
dtypes: int32(1), object(5)
memory usage: 239.5+ KB


,UF,COD. UF,COD. MUNIC,COD_IBGE,NOME DO MUNICÍPIO,POP_IBGE_2022
0,RO,11,00015,1100015,Alta Floresta D'Oeste,21558
1,RO,11,00023,1100023,Ariquemes,100896
2,RO,11,00031,1100031,Cabixi,5107
3,RO,11,00049,1100049,Cacoal,92202
4,RO,11,00056,1100056,Cerejeiras,15237


In [14]:
# Consolida df do IBGE
col_interesse1 = ['REGIAO', 'SIGLA_UF', 'NOME_UF', 'COD_IBGE', 'NOME_MUNICIPIO', 'LAT', 'LNG', 'POP_IBGE_2013', 'CAPITAL']
col_interesse2 = ['COD_IBGE', 'POP_IBGE_2022']
ibge3 = pd.merge(ibge1[col_interesse1],
                ibge2[col_interesse2],
                how='inner',
                left_on = 'COD_IBGE',
                right_on = 'COD_IBGE',
                indicator=True)

display(ibge3['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge']
for c in cols_delete:
    del ibge3[c] 

print('\nTotal de registros do ibge: ', ibge3.shape[0])
display(ibge3.head(5))

_merge
both          5570
left_only        0
right_only       0
Name: count, dtype: int64


Total de registros do ibge:  5570


,REGIAO,SIGLA_UF,NOME_UF,COD_IBGE,NOME_MUNICIPIO,LAT,LNG,POP_IBGE_2013,CAPITAL,POP_IBGE_2022
0,N,RO,Rondônia,1100015,Alta Floresta D'Oeste,-11.94,-62.00,25728,0,21558
1,N,RO,Rondônia,1100023,Ariquemes,-9.91,-63.03,101269,0,100896
2,N,RO,Rondônia,1100031,Cabixi,-13.50,-60.54,6495,0,5107
3,N,RO,Rondônia,1100049,Cacoal,-11.43,-61.44,85863,0,92202
4,N,RO,Rondônia,1100056,Cerejeiras,-13.20,-60.82,18041,0,15237


In [15]:
# para checar um municipio específico
# ibge[ibge['NOME_MUNICIPIO']=='Brasília']

## Carga dados IBGE - Regioes

Carga de csv com as meso e microrregiões.

- UF
- Nome_UF
- Cod_RG_Intermediaria
- Nome_RG_Intermediaria
- Cod_RG_Imediata
- Nome_RG_Imediata
- Cod_Mesorregiao
- Nome_Mesorregiao
- Cod_Microrregiao
- Nome_Microrregiao
- Cod_Mun
- Cod_IBGE
- Nome_Mun


In [16]:
# definir os tipos de algumas colunas
list_col_string = ['Cod_IBGE', 'Cod_Mesorregiao', 'Cod_Microrregiao'] 
dict_dtype = {column : 'str'  for column in list_col_string}

# dados atualizados de População (Censo 2022)
ibge_regioes = pd.read_csv('./dados/regioes_brasil.csv', 
                    sep='|', 
                    encoding='iso-8859-1',
                    dtype = dict_dtype, 
                    low_memory=False)

# trazer apenas colunas de interesse
ibge_regioes_res = ibge_regioes[['UF', 'Nome_UF', 'Cod_IBGE', 'Nome_Mun',
                             'Cod_Mesorregiao', 'Nome_Mesorregiao',
                             'Cod_Microrregiao','Nome_Microrregiao']]
ibge_regioes_res.head()

,UF,Nome_UF,Cod_IBGE,Nome_Mun,Cod_Mesorregiao,Nome_Mesorregiao,Cod_Microrregiao,Nome_Microrregiao
0,11,Rondônia,1100015,Alta Floresta D'Oeste,02,Leste Rondoniense,006,Cacoal
1,11,Rondônia,1100379,Alto Alegre dos Parecis,02,Leste Rondoniense,006,Cacoal
2,11,Rondônia,1100403,Alto Paraíso,02,Leste Rondoniense,003,Ariquemes
3,11,Rondônia,1100346,Alvorada D'Oeste,02,Leste Rondoniense,005,Alvorada D'Oeste
4,11,Rondônia,1100023,Ariquemes,02,Leste Rondoniense,003,Ariquemes


## Consolidar todos os dados IBGE

In [17]:
# Consolida df do IBGE com as Regioes
col_interesse = ['Cod_IBGE', 'Cod_Mesorregiao', 'Nome_Mesorregiao','Cod_Microrregiao','Nome_Microrregiao']

ibge = pd.merge(ibge3,
                ibge_regioes_res[col_interesse],
                how='inner',
                left_on = 'COD_IBGE',
                right_on = 'Cod_IBGE',
                indicator=True)

display(ibge['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge','Cod_IBGE']
for c in cols_delete:
    del ibge[c] 

print('\nTotal de registros do ibge: ', ibge.shape[0])
display(ibge.head(5))

_merge
both          5570
left_only        0
right_only       0
Name: count, dtype: int64


Total de registros do ibge:  5570


,REGIAO,SIGLA_UF,NOME_UF,COD_IBGE,NOME_MUNICIPIO,LAT,LNG,POP_IBGE_2013,CAPITAL,POP_IBGE_2022,Cod_Mesorregiao,Nome_Mesorregiao,Cod_Microrregiao,Nome_Microrregiao
0,N,RO,Rondônia,1100015,Alta Floresta D'Oeste,-11.94,-62.00,25728,0,21558,02,Leste Rondoniense,006,Cacoal
1,N,RO,Rondônia,1100023,Ariquemes,-9.91,-63.03,101269,0,100896,02,Leste Rondoniense,003,Ariquemes
2,N,RO,Rondônia,1100031,Cabixi,-13.50,-60.54,6495,0,5107,02,Leste Rondoniense,008,Colorado do Oeste
3,N,RO,Rondônia,1100049,Cacoal,-11.43,-61.44,85863,0,92202,02,Leste Rondoniense,006,Cacoal
4,N,RO,Rondônia,1100056,Cerejeiras,-13.20,-60.82,18041,0,15237,02,Leste Rondoniense,008,Colorado do Oeste


In [18]:
ibge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   REGIAO             5570 non-null   object 
 1   SIGLA_UF           5570 non-null   object 
 2   NOME_UF            5570 non-null   object 
 3   COD_IBGE           5570 non-null   object 
 4   NOME_MUNICIPIO     5570 non-null   object 
 5   LAT                5570 non-null   float64
 6   LNG                5570 non-null   float64
 7   POP_IBGE_2013      5570 non-null   int64  
 8   CAPITAL            5570 non-null   object 
 9   POP_IBGE_2022      5570 non-null   int32  
 10  Cod_Mesorregiao    5570 non-null   object 
 11  Nome_Mesorregiao   5570 non-null   object 
 12  Cod_Microrregiao   5570 non-null   object 
 13  Nome_Microrregiao  5570 non-null   object 
dtypes: float64(2), int32(1), int64(1), object(10)
memory usage: 587.6+ KB


### Salvar dados IBGE consolidados em CSV

In [19]:
# salvar em arquivo
# ibge.to_csv('./dados/preparados/dados_ibge_consolidado.csv', sep='|', index=False)

## Agrupar População e Qtd de Municipios por Estado

In [20]:
ibge_uf = ibge.groupby('SIGLA_UF').agg({'POP_IBGE_2022':'sum',
                                        'COD_IBGE':'count',
                                        'Cod_Mesorregiao': 'nunique',
                                        'Cod_Microrregiao': 'nunique'})
ibge_uf = ibge_uf.reset_index().rename(columns={'POP_IBGE_2022':'Total_Pop_IBGE_2022',
                                                'COD_IBGE':'Total_Mun_IBGE_2022',
                                                'Cod_Mesorregiao':'Total_Meso',
                                                'Cod_Microrregiao':'Total_Micro'})
ibge_uf.head()

,SIGLA_UF,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022,Total_Meso,Total_Micro
0,AC,829780,22,2,5
1,AL,3125254,102,3,13
2,AM,3952262,62,4,13
3,AP,774268,16,2,4
4,BA,14659023,417,7,32


In [21]:
# salvar CSV
#ibge_uf.to_csv('./dados/preparados/dados_ibge_agg_uf.csv', sep='|', index=False)

## Agrupar População e Qtd de Municipios por MesoRegiao

In [22]:
ibge_meso = ibge.groupby(['SIGLA_UF','Nome_Mesorregiao']).agg({'POP_IBGE_2022':'sum',
                                                               'COD_IBGE':'count'})
ibge_meso = ibge_meso.reset_index().rename(columns={'POP_IBGE_2022':'Total_Pop_IBGE_2022',
                                                    'COD_IBGE':'Total_Mun_IBGE_2022'})
ibge_meso.head()

,SIGLA_UF,Nome_Mesorregiao,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022
0,AC,Vale do Acre,583448,14
1,AC,Vale do Juruá,246332,8
2,AL,Agreste Alagoano,634079,24
3,AL,Leste Alagoano,2069392,52
4,AL,Sertão Alagoano,421783,26


In [23]:
# salvar CSV
#ibge_meso.to_csv('./dados/preparados/dados_ibge_agg_meso.csv', sep='|', index=False)

## Agrupar População e Qtd de Municipios por MicroRegiao

In [24]:
ibge_micro = ibge.groupby(['SIGLA_UF','Nome_Microrregiao']).agg({'POP_IBGE_2022':'sum',
                                                                 'COD_IBGE':'count'})
ibge_micro = ibge_micro.reset_index().rename(columns={'POP_IBGE_2022':'Total_Pop_IBGE_2022',
                                                      'COD_IBGE':'Total_Mun_IBGE_2022'})
ibge_micro.head(6)

,SIGLA_UF,Nome_Microrregiao,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022
0,AC,Brasiléia,70821,4
1,AC,Cruzeiro do Sul,157120,5
2,AC,Rio Branco,452963,7
3,AC,Sena Madureira,59664,3
4,AC,Tarauacá,89212,3
5,AL,Alagoana do Sertão do São Francisco,82149,3


In [25]:
# salvar CSV
#ibge_micro.to_csv('./dados/preparados/dados_ibge_agg_micro.csv', sep='|', index=False)

## Carga dados Indicadores Municipais (Atlas) - 2010

In [26]:
# definir os tipos de algumas colunas
list_col_string = ['Codmun6','Codmun7'] 
dict_dtype = {column : 'str'  for column in list_col_string}

df_ind_atlas = pd.read_csv('./dados/selecao_indicadores_sociais_municipios.csv', 
                       sep='|', dtype = dict_dtype, low_memory=False) 

# considerar apenas a ultima referencia (2010)
df_ind_atlas = df_ind_atlas[df_ind_atlas['ANO']==2010]

col_excluir = ['ANO']
for c in col_excluir: del df_ind_atlas[c]

# renomear colunas 
df_ind_atlas = df_ind_atlas.rename(columns = {
                                    'Codmun6':'CodIBGE_6',
                                    'Codmun7':'CodIBGE',
                                    'pesotot':'Pop_total',
                                    'pesourb':'Pop_urbana',
                                    'POP':'Pop_total_dom_part',    
                                    'POPT':'Pop_total_dom_part_exc_renda_nula'    
                                })

In [27]:
col_relevantes = ['CodIBGE', 'Pop_total','Pop_urbana','Pop_total_dom_part', 'Pop_total_dom_part_exc_renda_nula']
df_ind_atlas[col_relevantes].head(5).T

,11130,11131,11132,11133,11134
CodIBGE,1100015,1100023,1100031,1100049,1100056
Pop_total,24392,90353,6313,78574,17029
Pop_urbana,13970,76525,2693,61921,14419
Pop_total_dom_part,24097,89766,6303,78141,16942
Pop_total_dom_part_exc_renda_nula,22429,88730,6156,76876,16815


In [28]:
df_ind_atlas.shape

(5565, 111)

In [29]:
# checar as diferentes infos de Pop de um municipio
# display(ibge[ibge['NOME_MUNICIPIO']=='Brasília'])

# col = ['Município','Pop_total','Pop_urbana','Pop_total_dom_part','Pop_total_dom_part_exc_renda_nula']
# df_ind_atlas[df_ind_atlas['CodIBGE']=='5300108'][col]

## Carga dados das IES

In [30]:
%%time

# definir os tipos de algumas colunas 
# facilita operações de merge (tem que conter os mesmos tipos de dados)
list_col_string = ['CO_REGIAO_IES', 'CO_UF_IES', 'CO_MUNICIPIO_IES', 'CO_MESORREGIAO_IES', 
                   'CO_MICRORREGIAO_IES','CO_MANTENEDORA','CO_IES','NU_CEP_IES'] 
dict_dtype = {column : 'str'  for column in list_col_string}

df_ies_orig = pd.read_csv('./dados/MICRODADOS_ED_SUP_IES_2022.CSV', 
                          dtype = dict_dtype,
                          sep=';',
                          encoding='iso-8859-1',
                          low_memory=False)

print('Total registros: ', df_ies_orig.shape[0])
print('Total colunas: ', df_ies_orig.shape[1])

Total registros:  2595
Total colunas:  81
CPU times: total: 46.9 ms
Wall time: 58.2 ms


In [31]:
checar_df(df_ies_orig)

Informações do dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 81 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   NU_ANO_CENSO                    2595 non-null   int64 
 1   NO_REGIAO_IES                   2595 non-null   object
 2   CO_REGIAO_IES                   2595 non-null   object
 3   NO_UF_IES                       2595 non-null   object
 4   SG_UF_IES                       2595 non-null   object
 5   CO_UF_IES                       2595 non-null   object
 6   NO_MUNICIPIO_IES                2595 non-null   object
 7   CO_MUNICIPIO_IES                2595 non-null   object
 8   IN_CAPITAL_IES                  2595 non-null   int64 
 9   NO_MESORREGIAO_IES              2595 non-null   object
 10  CO_MESORREGIAO_IES              2595 non-null   object
 11  NO_MICRORREGIAO_IES             2595 non-null   object
 12  CO_MICRORREGIAO_IES  

,Contagem Unicos
CO_IES,2595
CO_MANTENEDORA,1769
CO_MESORREGIAO_IES,15
CO_MICRORREGIAO_IES,66
CO_MUNICIPIO_IES,719
CO_REGIAO_IES,5
CO_UF_IES,27
DS_COMPLEMENTO_ENDERECO_IES,893
DS_ENDERECO_IES,2217
DS_NUMERO_ENDERECO_IES,1311


Número de valores nulos:


,Contagem Nulos
CO_IES,0
CO_MANTENEDORA,0
CO_MESORREGIAO_IES,0
CO_MICRORREGIAO_IES,0
CO_MUNICIPIO_IES,0
CO_REGIAO_IES,0
CO_UF_IES,0
DS_COMPLEMENTO_ENDERECO_IES,1
DS_ENDERECO_IES,0
DS_NUMERO_ENDERECO_IES,1


# Consolidar dados no dataframe de IES

## Juntar dados IBGE

Aguns dados do dataframe ibge são adicionados ao dataframe de IES:
- POP_IBGE_2013
- POP_IBGE_2022
- CAPITAL
- LAT
- LNG
- Cod_Mesorregiao, Nome_Mesorregiao
- Cod_Microrregiao, Nome_Microrregiao

In [32]:
col_interesse = ['COD_IBGE',  'LAT', 'LNG', 'POP_IBGE_2013', 'POP_IBGE_2022', 'CAPITAL'] 
                #'Cod_Mesorregiao', 'Nome_Mesorregiao','Cod_Microrregiao','Nome_Microrregiao' - JA TEM, NAO PRECISA
       
df_ies_1 = pd.merge(df_ies_orig, 
                  ibge[col_interesse],
                  how='inner',
                  left_on = 'CO_MUNICIPIO_IES',
                  right_on = 'COD_IBGE',
                  indicator=True) 

display(df_ies_1['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge','CO_MUNICIPIO_IES']
for c in cols_delete:
    del df_ies_1[c] 

print('\nTotal de colunas do df_ies_orig: ', df_ies_orig.shape[1])
print('Total de colunas do df_ies_1: ', df_ies_1.shape[1])

_merge
both          2595
left_only        0
right_only       0
Name: count, dtype: int64


Total de colunas do df_ies_orig:  81
Total de colunas do df_ies_1:  86


## Criar df com contagem de meso e microrregioes com IES em cada UF

In [33]:
col_int = ['SG_UF_IES', 'NO_MUNICIPIO_IES', 'NO_MESORREGIAO_IES', 'NO_MICRORREGIAO_IES','NO_IES']
df_ies_1[col_int].head(5)

,SG_UF_IES,NO_MUNICIPIO_IES,NO_MESORREGIAO_IES,NO_MICRORREGIAO_IES,NO_IES
0,MT,Cuiabá,Centro-Sul Mato-grossense,Cuiabá,UNIVERSIDADE FEDERAL DE MATO GROSSO
1,MT,Cuiabá,Centro-Sul Mato-grossense,Cuiabá,UNIVERSIDADE DE CUIABÁ
2,MT,Cuiabá,Centro-Sul Mato-grossense,Cuiabá,CENTRO UNIVERSITÁRIO UNIC
3,MT,Cuiabá,Centro-Sul Mato-grossense,Cuiabá,FACULDADE CUIABÁ
4,MT,Cuiabá,Centro-Sul Mato-grossense,Cuiabá,INSTITUTO CUIABÁ DE ENSINO E CULTURA


In [34]:
# escolher uma UF
# AC: 12 IES; SE: 22
#df_ies_1[df_ies_1['SG_UF_IES']=='AC'][col_int] #12

df_ies_1[df_ies_1['SG_UF_IES']=='SE'][col_int]#.shape 

,SG_UF_IES,NO_MUNICIPIO_IES,NO_MESORREGIAO_IES,NO_MICRORREGIAO_IES,NO_IES
91,SE,São Cristóvão,Leste Sergipano,Aracaju,UNIVERSIDADE FEDERAL DE SERGIPE
1221,SE,Aracaju,Leste Sergipano,Aracaju,UNIVERSIDADE TIRADENTES
1222,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE PIO DÉCIMO
1223,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE SÃO LUÍS DE FRANÇA
1224,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE DE ADMINISTRAÇÃO E NEGÓCIOS DE SERGIPE
1225,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE BRASILEIRA DE TECNOLOGIA
1226,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE AMADEUS
1227,SE,Aracaju,Leste Sergipano,Aracaju,CENTRO UNIVERSITÁRIO ESTÁCIO DE SERGIPE
1228,SE,Aracaju,Leste Sergipano,Aracaju,FACULDADE SERGIPANA
1229,SE,Aracaju,Leste Sergipano,Aracaju,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SERGIPE"


In [35]:
contagem_regioes_UF_IES = df_ies_1.groupby('SG_UF_IES').agg({'NO_MESORREGIAO_IES':'nunique',
                                                             'NO_MICRORREGIAO_IES':'nunique'})
# checar exemplo para verificar se contagem esta correta
# contagem_regioes_UF_IES.loc['SE']

contagem_regioes_UF_IES = contagem_regioes_UF_IES.reset_index().rename(columns={'NO_MESORREGIAO_IES':'Total_Meso_IES',
                                                                                'NO_MICRORREGIAO_IES':'Total_Micro_IES'})

contagem_regioes_UF_IES.head()

,SG_UF_IES,Total_Meso_IES,Total_Micro_IES
0,AC,2,2
1,AL,3,5
2,AM,1,5
3,AP,1,1
4,BA,7,24


In [36]:
# checar um exemplo
#contagem_regioes_UF_IES[contagem_regioes_UF_IES['SG_UF_IES']=='SE']

## Juntar dados de indicadores municipais

Alguns dados de indicadores de unicipios são adicionados ao dataframe de IES:
- **Pop_total** (2010): População total
- **Pop_urbana**: População urbana 
- **Pop_total_dom_part**: População total em domicílios particulares permanentes
- **Pop_total_dom_part_exc_renda_nula**: População total em domicílios particulares permanentes, exceto com renda nula
  

In [37]:
col_interesse = ['CodIBGE', 'Pop_total','Pop_urbana','Pop_total_dom_part', 'Pop_total_dom_part_exc_renda_nula']
       
df_ies = pd.merge(df_ies_1, 
                  df_ind_atlas[col_interesse],
                  how='inner',
                  left_on = 'COD_IBGE',
                  right_on = 'CodIBGE',
                  indicator=True
                 ) 

display(df_ies['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge', 'CodIBGE']
for c in cols_delete:
    del df_ies[c] 

print('\nTotal de colunas do df_ies (antes): ', df_ies_1.shape[1])
print('Total de colunas do df_ies: ', df_ies.shape[1])

_merge
both          2595
left_only        0
right_only       0
Name: count, dtype: int64


Total de colunas do df_ies (antes):  86
Total de colunas do df_ies:  90


# Tratamento dados 

## Enriquecimento de dados

### Adicionar Descrição Tipo Organização Acadêmica

In [38]:
# Convertendo coluna TP_ORGANIZACAO_ACADEMICA
df_ies.insert(df_ies.columns.get_loc('TP_ORGANIZACAO_ACADEMICA')+1, \
                'Tipo_Org_Acad', df_ies['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_acad(x)))

In [39]:
df_ies['Tipo_Org_Acad'].value_counts()

Tipo_Org_Acad
Faculdade                                              1968
Centro Universitário                                    381
Universidade                                            205
Instituto Federal de Educação, Ciência e Tecnologia      39
Centro Federal de Educação Tecnológica                    2
Name: count, dtype: int64

In [40]:
df_ies['Tipo_Org_Acad'].value_counts(normalize=True)

Tipo_Org_Acad
Faculdade                                             0.76
Centro Universitário                                  0.15
Universidade                                          0.08
Instituto Federal de Educação, Ciência e Tecnologia   0.02
Centro Federal de Educação Tecnológica                0.00
Name: proportion, dtype: float64

### Adicionar Organizacao principal: Faculdade, Centro e Outros 

In [41]:
# Simplificando o conceito de Organização administrativa
df_ies.insert(df_ies.columns.get_loc('Tipo_Org_Acad')+1, \
                'Tipo_Org_Principal', df_ies['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_principal(x)))

In [42]:
df_ies['Tipo_Org_Principal'].value_counts(normalize=True)

Tipo_Org_Principal
Faculdade              0.84
Centro Universitário   0.15
Outros                 0.02
Name: proportion, dtype: float64

### Adicionar Descrição Tipo Categoria Administrativa

In [43]:
# Convertendo coluna TP_CATEGORIA_ADMINISTRATIVA
df_ies.insert(df_ies.columns.get_loc('TP_CATEGORIA_ADMINISTRATIVA')+1, \
                'Tipo_Cat_Admn', df_ies['TP_CATEGORIA_ADMINISTRATIVA'].apply(lambda x: define_tp_cat_adm(x)))

### Adicionar Tipo: Pública ou privada

In [44]:
df_ies['Tipo_Cat_Admn'].value_counts()

Tipo_Cat_Admn
Privada com fins lucrativos    1449
Privada sem fins lucrativos     834
Pública Estadual                133
Pública Federal                 120
Pública Municipal                34
Especial                         25
Name: count, dtype: int64

In [45]:
df_ies['TIPO_INST'] = np.where(df_ies['Tipo_Cat_Admn'].str.contains('Pública'),'Pública','Privada')
df_ies['TIPO_INST'].value_counts()

TIPO_INST
Privada    2308
Pública     287
Name: count, dtype: int64

# Verificação final - dataframe de IES

In [46]:
colunas = ['NO_IES', 'SG_UF_IES','NO_REGIAO_IES', 'NO_MUNICIPIO_IES', 'COD_IBGE',
           'NO_MANTENEDORA', 
           'Tipo_Org_Acad', 'Tipo_Cat_Admn', 'TIPO_INST']
df_ies[colunas].head().T

,0,1,2,3,4
NO_IES,UNIVERSIDADE FEDERAL DE MATO GROSSO,UNIVERSIDADE DE CUIABÁ,CENTRO UNIVERSITÁRIO UNIC,FACULDADE CUIABÁ,INSTITUTO CUIABÁ DE ENSINO E CULTURA
SG_UF_IES,MT,MT,MT,MT,MT
NO_REGIAO_IES,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste,Centro-Oeste
NO_MUNICIPIO_IES,Cuiabá,Cuiabá,Cuiabá,Cuiabá,Cuiabá
COD_IBGE,5103403,5103403,5103403,5103403,5103403
NO_MANTENEDORA,FUNDACAO UNIVERSIDADE FEDERAL DE MATO GROSSO,PITAGORAS - SISTEMA DE EDUCACAO SUPERIOR SOCIEDADE LTDA,EDITORA E DISTRIBUIDORA EDUCACIONAL S/A,ASSOCIACAO DOM AQUINO CORREA - ADAC,ASSUPERO ENSINO SUPERIOR LTDA.
Tipo_Org_Acad,Universidade,Universidade,Centro Universitário,Faculdade,Faculdade
Tipo_Cat_Admn,Pública Federal,Privada com fins lucrativos,Privada com fins lucrativos,Privada sem fins lucrativos,Privada com fins lucrativos
TIPO_INST,Pública,Privada,Privada,Privada,Privada


In [47]:
checar_df(df_ies)

Informações do dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 94 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   NU_ANO_CENSO                       2595 non-null   int64  
 1   NO_REGIAO_IES                      2595 non-null   object 
 2   CO_REGIAO_IES                      2595 non-null   object 
 3   NO_UF_IES                          2595 non-null   object 
 4   SG_UF_IES                          2595 non-null   object 
 5   CO_UF_IES                          2595 non-null   object 
 6   NO_MUNICIPIO_IES                   2595 non-null   object 
 7   IN_CAPITAL_IES                     2595 non-null   int64  
 8   NO_MESORREGIAO_IES                 2595 non-null   object 
 9   CO_MESORREGIAO_IES                 2595 non-null   object 
 10  NO_MICRORREGIAO_IES                2595 non-null   object 
 11  CO_MICRORREGIAO_IES          

,Contagem Unicos
CAPITAL,2
COD_IBGE,719
CO_IES,2595
CO_MANTENEDORA,1769
CO_MESORREGIAO_IES,15
CO_MICRORREGIAO_IES,66
CO_REGIAO_IES,5
CO_UF_IES,27
DS_COMPLEMENTO_ENDERECO_IES,893
DS_ENDERECO_IES,2217


Número de valores nulos:


,Contagem Nulos
CAPITAL,0
COD_IBGE,0
CO_IES,0
CO_MANTENEDORA,0
CO_MESORREGIAO_IES,0
CO_MICRORREGIAO_IES,0
CO_REGIAO_IES,0
CO_UF_IES,0
DS_COMPLEMENTO_ENDERECO_IES,1
DS_ENDERECO_IES,0


## Salvar dados IES consolidados em CSV

In [48]:
# salvar em arquivo
#df_ies.to_csv('./dados/preparados/dados_ies_consolidado.csv', sep='|', index=False)

# Criação Dataframes específicos 

## Distribuição IES - varias agregações por Estado

Vários dataframes temporários serão criados e depois consolidados em um dataframe principal (df1).

df1: Agregações por UF
- Total_IES: total de IES na UF
- Total_mun: total de municipios com existencia de IES
- Total_Pop_IES: total da população de municipios com existencia de IES
- Total_Priv: total de IES privadas na UF
- Total_Publ: total de IES publicas na UF
- Total_Pop_IBGE_2022: total da população de todos os municipios
- Total_Mun_IBGE_2022: total de municipios da UF
- Prop_Mun: proporção - total de municipios com de IES/ total de municipios da UF (cobertura de IES)
- IES_hab: Total_Pop_IES / Total_IES

In [49]:
# Numero de IES por Estado
# Numero de municipios por Estado (apenas aqueles que contém ao menos uma IES)
colunas = ['NO_IES', 'SG_UF_IES','NO_REGIAO_IES', 'NO_MUNICIPIO_IES', 'COD_IBGE',
           'NO_MANTENEDORA', 
           'Tipo_Org_Acad', 'Tipo_Cat_Admn','TIPO_INST',
           'POP_IBGE_2013', 'POP_IBGE_2022','Pop_total','Pop_urbana',
           'Pop_total_dom_part','Pop_total_dom_part_exc_renda_nula']

t1 = df_ies[colunas].copy()
t1 = t1.groupby('SG_UF_IES').agg({'NO_IES':'count', 'COD_IBGE':'nunique'})
#t1.head()

In [50]:
#df_ies[df_ies['SG_UF_IES']=='AC'].shape #12 IES

In [51]:
# População de cada Estado, considerando população dos municípios com IES
t2 = df_ies[colunas].copy()
t2 = t2.groupby(['SG_UF_IES','COD_IBGE'])['POP_IBGE_2022'].max().reset_index()
t2 = t2.groupby('SG_UF_IES')['POP_IBGE_2022'].sum().to_frame()
#t2.head()

In [52]:
# Numero de IES publicas e privadas por Estado
t3 = df_ies[colunas].copy()
t3 = t3.groupby(['SG_UF_IES','TIPO_INST'])['NO_IES'].count().reset_index().rename(columns={'NO_IES':'Total_inst'})
t3 = pd.pivot(t3, index='SG_UF_IES', columns='TIPO_INST', values = 'Total_inst')
#t3.head()

In [53]:
lista_df = [t1, t2, t3]
df1_temp = pd.concat(lista_df, axis=1, join='inner', ignore_index=False).reset_index()
df1_temp.columns = ['SG_UF_IES', 'Total_IES','Total_mun','Total_Pop_IES','Total_Priv','Total_Publ'] 

# incluir outras infos por UF ao df1
# Total_Pop_IBGE_2022 	Total_Mun_IBGE_2022 	Total_Meso 	Total_Micro
df1_temp = pd.merge(df1_temp,
                ibge_uf,
                left_on = 'SG_UF_IES',
                right_on = 'SIGLA_UF',
                how='inner',
                indicator=True)

display(df1_temp['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge','SIGLA_UF']
for c in cols_delete:
    del df1_temp[c] 

# Qtde de municipios com IES / Total municipios
# possibilita grafico - distribuição dos Municípios com IES por Estados
df1_temp['Prop_Mun'] = df1_temp['Total_mun'] / df1_temp['Total_Mun_IBGE_2022']*100

# Proporção Qtd População / Total IES
df1_temp['IES_hab'] = df1_temp['Total_Pop_IES'] / df1_temp['Total_IES']

df1_temp.head()

_merge
both          27
left_only      0
right_only     0
Name: count, dtype: int64

,SG_UF_IES,Total_IES,Total_mun,Total_Pop_IES,Total_Priv,Total_Publ,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022,Total_Meso,Total_Micro,Prop_Mun,IES_hab
0,AC,12,2,458713,10,2,829780,22,2,5,9.09,38226.08
1,AL,33,6,1357048,29,4,3125254,102,3,13,5.88,41122.67
2,AM,32,6,2502816,29,3,3952262,62,4,13,9.68,78213.00
3,AP,16,2,593443,13,3,774268,16,2,4,12.50,37090.19
4,BA,153,44,7596531,143,10,14659023,417,7,32,10.55,49650.53


In [54]:
# acrescentar dados contagem_regioes_UF_IES
df1 = pd.merge(df1_temp,
               contagem_regioes_UF_IES,
               how='inner',
               left_on = ['SG_UF_IES'],
               right_on = ['SG_UF_IES'],
               indicator=True)

display(df1['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge']
for c in cols_delete:
    del df1[c] 


_merge
both          27
left_only      0
right_only     0
Name: count, dtype: int64

In [55]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SG_UF_IES            27 non-null     object 
 1   Total_IES            27 non-null     int64  
 2   Total_mun            27 non-null     int64  
 3   Total_Pop_IES        27 non-null     int32  
 4   Total_Priv           27 non-null     int64  
 5   Total_Publ           27 non-null     int64  
 6   Total_Pop_IBGE_2022  27 non-null     int32  
 7   Total_Mun_IBGE_2022  27 non-null     int64  
 8   Total_Meso           27 non-null     int64  
 9   Total_Micro          27 non-null     int64  
 10  Prop_Mun             27 non-null     float64
 11  IES_hab              27 non-null     float64
 12  Total_Meso_IES       27 non-null     int64  
 13  Total_Micro_IES      27 non-null     int64  
dtypes: float64(2), int32(2), int64(9), object(1)
memory usage: 2.9+ KB


In [56]:
df1.sort_values(by='Prop_Mun', ascending=False).head(10)

,SG_UF_IES,Total_IES,Total_mun,Total_Pop_IES,Total_Priv,Total_Publ,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022,Total_Meso,Total_Micro,Prop_Mun,IES_hab,Total_Meso_IES,Total_Micro_IES
6,DF,66,1,2923369,61,5,2923369,1,1,1,100.00,44293.47,1,1
18,RJ,130,29,13818111,101,29,16615526,92,6,18,31.52,106293.16,6,13
7,ES,71,19,2894544,66,5,3975100,78,4,13,24.36,40768.23,4,10
25,SP,585,152,38664837,481,104,46024937,645,15,63,23.57,66093.74,15,58
20,RO,32,10,1081940,30,2,1616379,52,2,8,19.23,33810.62,2,6
12,MT,72,24,2487056,68,4,3784239,141,5,22,17.02,34542.44,5,15
5,CE,103,29,5186911,96,7,8936431,184,7,33,15.76,50358.36,7,18
15,PE,114,29,5403681,106,8,9051113,185,5,19,15.68,47400.71,5,16
8,GO,118,37,4849970,108,10,6950976,246,5,18,15.04,41101.44,5,14
13,PA,78,21,4334465,72,6,8442962,144,6,22,14.58,55570.06,6,15


In [57]:
# Acrescentar dados
df1['Cob_Meso'] = df1['Total_Meso_IES'] / df1['Total_Meso'] * 100
df1['Cob_Micro'] = df1['Total_Micro_IES'] / df1['Total_Micro'] * 100

### Checar casos especificos

In [58]:
df1[df1['SG_UF_IES']=='RS'].T

,22
SG_UF_IES,RS
Total_IES,118
Total_mun,36
Total_Pop_IES,5661809
Total_Priv,107
Total_Publ,11
Total_Pop_IBGE_2022,11088065
Total_Mun_IBGE_2022,497
Total_Meso,7
Total_Micro,35


### Salvar dados IES com agregaçoes por UF em CSV

In [59]:
# salvar em arquivo
#df1.to_csv('./dados/preparados/dados_IES_agg_UF.csv', sep='|', index=False)

In [60]:
df_ies[['CO_IES','NO_IES']].sample(20)

,CO_IES,NO_IES
197,5403,CENTRO UNIVERSITÁRIO OPET
1778,19786,FACULDADE ANHANGUERA DE SERRA
2580,24255,Faculdade de Nova Mutum
1984,3157,INSTITUTO SUPERIOR DE EDUCAÇÃO ALMEIDA RODRIGUES
640,1637,FACULDADE DELMIRO GOUVEIA
1256,1913,CENTRO UNIVERSITÁRIO FAMINAS
1291,994,Faculdade de Duque de Caxias
1072,2652,FACULDADE UNIDA DE VITÓRIA
941,16879,Faculdade de Medicina de Olinda
1374,4865,FACULDADE ANHANGUERA DE SÃO LUIZ


## Distribuição IES - agregações por Regiao (IGNORAR)

df2: Agregações por Regiao
- Total_IES: total de IES em cada Regiao
- Total_mun: total de municipios com existencia de IES
- Total_Priv: total de IES privadas na UF
- Total_Publ: total de IES publicas na UF


In [61]:
# Numero de IES por Regiao
colunas = ['NO_IES', 'SG_UF_IES', 'NO_MUNICIPIO_IES', 'COD_IBGE',
            'NO_REGIAO_IES', 'NO_MESORREGIAO_IES', 'NO_MICRORREGIAO_IES',
            'NO_MANTENEDORA', 'Tipo_Org_Acad', 'Tipo_Cat_Admn','TIPO_INST',
            'POP_IBGE_2013', 'POP_IBGE_2022','Pop_total','Pop_urbana',
           'Pop_total_dom_part','Pop_total_dom_part_exc_renda_nula']

In [62]:
t4 = df_ies[colunas].copy()
t4 = t4.groupby(['SG_UF_IES','NO_MESORREGIAO_IES']).agg({'NO_IES':'count',
                                                    'COD_IBGE':'nunique'})
t4.head(5)

NO_IES  COD_IBGE
SG_UF_IES NO_MESORREGIAO_IES                  
AC        Vale do Acre            10         1
          Vale do Juruá            2         1
AL        Agreste Alagoano        10         3
          Leste Alagoano          22         2
          Sertão Alagoano          1         1

In [63]:
#para checar
#df_ies[df_ies['SG_UF_IES']=='AL'][['NO_MESORREGIAO_IES','NO_MUNICIPIO_IES']].sort_values(by='NO_MESORREGIAO_IES')

In [64]:
# Numero de IES publicas e privadas por Estado
t5 = df_ies[colunas].copy()
t5 = t5.groupby(['SG_UF_IES','NO_MESORREGIAO_IES','TIPO_INST'])['NO_IES'].count().reset_index()
t5 = t5.rename(columns = {'NO_IES':'Total_inst'})
t5 = pd.pivot(t5, index=['SG_UF_IES','NO_MESORREGIAO_IES'], columns = 'TIPO_INST', values='Total_inst')
t5['Privada'] = t5['Privada'].replace(np.nan, 0)
t5['Pública'] = t5['Pública'].replace(np.nan, 0)
t5.head()

TIPO_INST                     Privada  Pública
SG_UF_IES NO_MESORREGIAO_IES                  
AC        Vale do Acre           8.00     2.00
          Vale do Juruá          2.00     0.00
AL        Agreste Alagoano       9.00     1.00
          Leste Alagoano        19.00     3.00
          Sertão Alagoano        1.00     0.00

In [65]:
# Para checar
#df_ies[df_ies['SG_UF_IES']=='AC'][['NO_MESORREGIAO_IES','NO_IES','TIPO_INST']]

In [66]:
lista_df = [t4, t5]
df2 = pd.concat(lista_df, axis=1, join='inner', ignore_index=False).reset_index()
df2.columns = ['SG_UF_IES', 'NO_MESORREGIAO_IES', 'Total_IES','Total_mun','Total_Priv','Total_Publ'] 

# incluir outras infos ao df2: Total_Mun_IBGE_2022 	
df2 = pd.merge(df2,
                ibge_meso[['SIGLA_UF','Nome_Mesorregiao', 'Total_Mun_IBGE_2022']],
                left_on = ['SG_UF_IES','NO_MESORREGIAO_IES'],
                right_on = ['SIGLA_UF','Nome_Mesorregiao'],
                how='inner',
                indicator=True)

display(df2['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge', 'SIGLA_UF','NO_MESORREGIAO_IES']
for c in cols_delete:
    del df2[c] 

# Qtde de municipios com IES / Total municipios
# possibilita grafico - distribuição dos Municípios com IES por Estados
df2['Prop_Mun'] = df2['Total_mun'] / df2['Total_Mun_IBGE_2022']*100

df2.head()

_merge
both          131
left_only       0
right_only      0
Name: count, dtype: int64

,SG_UF_IES,Total_IES,Total_mun,Total_Priv,Total_Publ,Nome_Mesorregiao,Total_Mun_IBGE_2022,Prop_Mun
0,AC,10,1,8.00,2.00,Vale do Acre,14,7.14
1,AC,2,1,2.00,0.00,Vale do Juruá,8,12.50
2,AL,10,3,9.00,1.00,Agreste Alagoano,24,12.50
3,AL,22,2,19.00,3.00,Leste Alagoano,52,3.85
4,AL,1,1,1.00,0.00,Sertão Alagoano,26,3.85


In [67]:
# juntar população de cada regiao
df2 = pd.merge(df2,
               ibge_meso[['SIGLA_UF','Nome_Mesorregiao','Total_Pop_IBGE_2022']],
               left_on = ['SG_UF_IES','Nome_Mesorregiao'],
               right_on = ['SIGLA_UF','Nome_Mesorregiao'],
               how='inner',
               indicator=True)

display(df2['_merge'].value_counts()) # para assegurar que todos os dados foram preenchidos

cols_delete = ['_merge', 'SIGLA_UF']
for c in cols_delete:
    del df2[c] 

# reorganizar as colunas
df2 = df2[['SG_UF_IES','Nome_Mesorregiao',
           'Total_IES', 'Total_mun', 'Total_Priv', 'Total_Publ',
           'Total_Pop_IBGE_2022', 'Total_Mun_IBGE_2022']].copy()

df2.head()

_merge
both          131
left_only       0
right_only      0
Name: count, dtype: int64

,SG_UF_IES,Nome_Mesorregiao,Total_IES,Total_mun,Total_Priv,Total_Publ,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022
0,AC,Vale do Acre,10,1,8.00,2.00,583448,14
1,AC,Vale do Juruá,2,1,2.00,0.00,246332,8
2,AL,Agreste Alagoano,10,3,9.00,1.00,634079,24
3,AL,Leste Alagoano,22,2,19.00,3.00,2069392,52
4,AL,Sertão Alagoano,1,1,1.00,0.00,421783,26


In [68]:
#df2[df2['SG_UF_IES'].isin(['MA'])]

In [69]:
df1[df1['SG_UF_IES'].isin(['MG','SP','MA'])]

,SG_UF_IES,Total_IES,Total_mun,Total_Pop_IES,Total_Priv,Total_Publ,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022,Total_Meso,Total_Micro,Prop_Mun,IES_hab,Total_Meso_IES,Total_Micro_IES,Cob_Meso,Cob_Micro
9,MA,66,21,2866300,62,4,6800605,217,5,21,9.68,43428.79,5,12,100.00,57.14
10,MG,312,106,13084266,290,22,20732660,853,12,66,12.43,41936.75,12,57,100.00,86.36
25,SP,585,152,38664837,481,104,46024937,645,15,63,23.57,66093.74,15,58,100.00,92.06


In [70]:
df1.sort_values(by='Total_Pop_IES', ascending=False).head(10)

,SG_UF_IES,Total_IES,Total_mun,Total_Pop_IES,Total_Priv,Total_Publ,Total_Pop_IBGE_2022,Total_Mun_IBGE_2022,Total_Meso,Total_Micro,Prop_Mun,IES_hab,Total_Meso_IES,Total_Micro_IES,Cob_Meso,Cob_Micro
25,SP,585,152,38664837,481,104,46024937,645,15,63,23.57,66093.74,15,58,100.00,92.06
18,RJ,130,29,13818111,101,29,16615526,92,6,18,31.52,106293.16,6,13,100.00,72.22
10,MG,312,106,13084266,290,22,20732660,853,12,66,12.43,41936.75,12,57,100.00,86.36
17,PR,172,56,7711938,158,14,11835379,399,10,39,14.04,44836.85,10,29,100.00,74.36
4,BA,153,44,7596531,143,10,14659023,417,7,32,10.55,49650.53,7,24,100.00,75.00
22,RS,118,36,5661809,107,11,11088065,497,7,35,7.24,47981.43,7,21,100.00,60.00
15,PE,114,29,5403681,106,8,9051113,185,5,19,15.68,47400.71,5,16,100.00,84.21
5,CE,103,29,5186911,96,7,8936431,184,7,33,15.76,50358.36,7,18,100.00,54.55
8,GO,118,37,4849970,108,10,6950976,246,5,18,15.04,41101.44,5,14,100.00,77.78
23,SC,106,33,4633643,98,8,7762154,295,6,20,11.19,43713.61,6,16,100.00,80.00
